In [3]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

# Project modules
from utils import *
from utils_visualisation import *
from Data_Handler import ClimbingDataset
from models import *
from trainer import *

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torchmetrics.classification import MulticlassAccuracy


# Image filters
import kornia.filters as KF

#Bayesian Hyperparam
import optuna

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
torch.__version__
device = "cuda" if torch.cuda.is_available() else "cpu"
device


'cpu'

In [ ]:
def objective(trial: optuna.Trial):
    blur_kernel_size = trial.suggest_categorical("blur_kernel_size", [1, 3, 5, 7]) 
    blur_sigma = trial.suggest_float("blur_sigma", 0.5, 3)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
    hidden_units_CNN = trial.suggest_categorical("hidden_units_CNN", [8, 16, 32, 64])
    hidden_units_classifier = trial.suggest_categorical("hidden_units_classifier", [16, 32, 64, 128, 256])
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)   # log scale
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    
    # Load Data 
    # necessary bc blur is hyperparameter dependent
    
    transform = KF.GaussianBlur2d(kernel_size=(blur_kernel_size, blur_kernel_size), sigma=(blur_sigma, blur_sigma), border_type="constant")
    two_class_dataset = ClimbingDataset(board_names=["12 x 12 with kickboard Square"],
                                map=True, 
                                transform=transform,
                                label_filter=[5, 14],  # 6a+ & 7c
                                max_samples=10000 
    )
    num_classes = two_class_dataset.num_classes
    
    
    train_size = int(0.8 * len(two_class_dataset))  # 80% train
    test_size = len(two_class_dataset) - train_size

    # Deterministic split (set generator seed for reproducibility)
    train_dataset, test_dataset = random_split(
        two_class_dataset,
        [train_size, test_size],
        generator=torch.Generator().manual_seed(42)
    )

    # Wrap in DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
        
    
    #Model
    model_0 = shallowCNN(input_shape= 4,
                     hidden_units_CNN= hidden_units_CNN, 
                     hidden_units_classifier=  hidden_units_classifier, 
                     output_shape=num_classes 
                     ).to(device)


    # Loss and Optimizer
    loss_fn = torch.nn.BCEWithLogitsLoss()
    acc_fn = Multiclass_accuracy()# MulticlassAccuracy(num_classes=num_classes).to(device)
    optimizer = optim.Adam(model_0.parameters(), lr=lr, weight_decay=weight_decay )
    
    
    model, history = train_model(
        model=model_0,
        train_loader=train_loader,
        test_loader=test_loader,
        loss_fn=loss_fn,
        optimizer=optimizer,
        acc_fn=acc_fn,
        device=device,
        epochs=30,
        patience=7
    )
    
    return history["test_acc"][-1]
    
    
    
    



In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=5)

print("Best trial:")
print("  Value:", study.best_trial.value)
print("  Params:", study.best_trial.params)

[I 2025-09-26 21:15:50,161] A new study created in memory with name: no-name-92039262-692c-4919-abe7-6c1488382859


/home/fillies/Documents/moon/kilter/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
[I 2025-09-26 21:16:06,826] Trial 0 finished with value: 0.7531994049038205 and parameters: {'blur_kernel_size': 3, 'blur_sigma': 2.224280366857611, 'batch_size': 16, 'hidden_units_CNN': 16, 'hidden_units_classifier': 32, 'lr': 0.007573217737704389, 'weight_decay': 2.1844583478564822e-06}. Best is trial 0 with value: 0.7531994049038205.
/home/fillies/Documents/moon/kilter/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
[I 2025-09-26 21:16:38,525] Trial 1 finished with value: 0.42991071433893274 and parameters: {'blur_kernel_size': 5, 'blur_sigma': 1.3085288000054336, 'bat